In [ ]:
!pip install langgraph langsmith langchain langchain_groq langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.3/423.3 kB 28.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.2/47.2 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.6/223.6 kB 9.9 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.50
    Uninstalling langchain-core-0.3.

In [ ]:
from langchain_groq import ChatGroq
from google.colab import userdata
groq_api_key = userdata.get('groq_api_key')

In [ ]:
from typing import Dict, List, TypedDict, Annotated, Sequence, Any
from langgraph.graph import StateGraph, END
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, ToolMessage
from langgraph.prebuilt import ToolNode, tools_condition
import operator
from langchain_core.documents import Document
from langchain_core.runnables import RunnableConfig
import json
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
import re

from langchain_core.pydantic_v1 import BaseModel, Field
from typing_extensions import Literal
from langchain_core.prompts import ChatPromptTemplate

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3553: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
class GraphState(TypedDict):  # Class GraphState
    messages: Annotated[Sequence[BaseMessage], operator.add]
    current_agent: str
    current_llm_model: str
    context_summary: str

state = StateGraph(GraphState)

In [ ]:
prompt_generator = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
system = """You are a prompt generator for text to image model. Your task is to generate 50 diffrent prompts for text to image model to generate diverse images given to you
the scenarios. example if I give you scenario as 'people in distress in the context to disaster' you give me different prompts for image generator model to
generate a complete dataset of images of people in distress covering diverse scenarios. """
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

prompt_generator_agent = prompt | prompt_generator

def prompt_generator_handler(state: GraphState):
    """ The marketing RAG powered Agent to resolve user queries"""

    question = state["messages"][-1].content

    inbuilt_query = f""" You are a prompt generator for a text-to-image model. Given the user scenario and access to the full conversation history,
your task is to generate 50 diverse and creative image generation prompts based on the scenario provided by the user.
Make sure the prompts cover a wide range of sub-scenarios, perspectives, environments, and subjects to create a comprehensive dataset of images.
what i need as the output exactly is: return the python list of 50 prompt strings. agent_output.content should be a string like: '["prompt1", "prompt2", ...]'

User scenario:
{question}

    """
    response = prompt_generator_agent.invoke(inbuilt_query)

    # Add the question-response pair to conversation_history
    conversation_id = len(state["conversation_history"]) + 1
    state["conversation_history"][conversation_id] = {
        "question": question,
        "response": response
    }

    state["llm_model"] = "Gemma2-9b-It"

    # Also add the AI response to the messages list
    state["messages"].append(AIMessage(content=response.content))

    # Update the current_agent within the state dictionary
    state['current_agent'] = "prompt_generator_agent"  # Or the appropriate agent name

    # Return the updated state
    return state


In [ ]:
chat_generator = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
system = """You are a prompt generator for text to image model. Your task is to generate 50 diffrent prompts for text to image model to generate diverse images given to you
the scenarios. example if I give you scenario as 'people in distress in the context to disaster' you give me different prompts for image generator model to
generate a complete dataset of images of people in distress covering diverse scenarios. """
chat_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

chat_agent = chat_prompt | chat_generator

def chat_handler(state: GraphState):
    """ The marketing RAG powered Agent to resolve user queries"""

    question = state["messages"][-1].content

    inbuilt_query = f""" You are a prompt generator for a text-to-image model. Given the user scenario and access to the full conversation history,
your task is to generate 50 diverse and creative image generation prompts based on the scenario provided by the user.
Make sure the prompts cover a wide range of sub-scenarios, perspectives, environments, and subjects to create a comprehensive dataset of images.

User scenario:
{question}
    """
    response = prompt_generator_agent.invoke(inbuilt_query)

    # Add the question-response pair to conversation_history
    conversation_id = len(state["conversation_history"]) + 1
    state["conversation_history"][conversation_id] = {
        "question": question,
        "response": response
    }

    state["llm_model"] = "Gemma2-9b-It"

    # Also add the AI response to the messages list
    state["messages"].append(AIMessage(content=response.content))

    # Update the current_agent within the state dictionary
    state['current_agent'] = "chat_agent"  # Or the appropriate agent name

    # Return the updated state
    return state


In [ ]:
def main():
    # Initialize the state
    initial_state = {
        "messages": [],
        "current_agent": "prompt_generator_agent",  # Set initial agent
        "current_llm_model": "deepseek-r1-distill-qwen-32b",
        "context_summary": "",
        "conversation_history": {} # Initialize conversation history
    }

    while True:
        user_input = input("Enter a scenario (or type 'exit' to quit): ")
        if user_input.lower() == 'exit':
            break

        # Create a HumanMessage from the input
        initial_state["messages"].append(HumanMessage(content=user_input))

        # Invoke the prompt_generator_handler
        updated_state = prompt_generator_handler(initial_state)

        # Print or process the updated state's AI response
        print(updated_state["messages"][-1].content)

        # Update the initial state with the new state for the next iteration
        initial_state = updated_state

if __name__ == "__main__":
    main()


Enter a scenario (or type 'exit' to quit): Birds
```python
agent_output = [
    "A majestic bald eagle soaring above snow-capped mountains at sunset",
    "A hummingbird hovering in mid-air, sipping nectar from a vibrant flower",
    "A flock of colorful parrots squawking in a lush rainforest canopy",
    "A lone penguin waddling across a frozen Antarctic landscape",
    "A pair of lovebirds perched on a tree branch, cuddling each other",
    "A tiny wren singing a beautiful melody in a dew-kissed garden",
    "A peacock spreading its iridescent tail feathers in a display of grandeur",
    "A group of owls perched on branches, their glowing eyes observing the night",
    "A majestic owl with its wings spread wide, silhouetted against a full moon",
    "A cardinal perched on a snow-covered branch, its bright red plumage contrasting with the white",
    "A flock of starlings flying in a mesmerizing V-formation across a clear blue sky",
    "A blue jay perched on a fence post, its blue an

In [ ]:
import openai
import requests
import os
import ast
from time import sleep

# Set OpenAI API key from Colab's env
# openai.api_key = os.environ["OPENAI_API_KEY"]

# Directory to save images
output_dir = "/content/generated_images"
os.makedirs(output_dir, exist_ok=True)

initial_state = {
        "messages": [],
        "current_agent": "prompt_generator_agent",  # Set initial agent
        "current_llm_model": "deepseek-r1-distill-qwen-32b",
        "context_summary": "",
        "conversation_history": {} # Initialize conversation history
    }

# === 🔁 STEP 1: Invoke your prompt generator agent ===
scenario = input("Enter a scenario (or type 'exit' to quit): ")
# agent_output = prompt_generator_agent.invoke(scenario)

initial_state["messages"].append(HumanMessage(content=scenario))

# Invoke the prompt_generator_handler
updated_state = prompt_generator_handler(initial_state)

# Print or process the updated state's AI response
agent_output = updated_state["messages"][-1].content
print(updated_state["messages"][-1].content)

# Update the initial state with the new state for the next iteration
initial_state = updated_state


# === 🧠 STEP 2: Parse output string to Python list ===
# agent_output.content should be a string like: '["prompt1", "prompt2", ...]'


try:
    # Attempt to parse as JSON first
    prompt_list = json.loads(agent_output)

    # If JSON parsing fails, fallback to ast.literal_eval
except json.JSONDecodeError:
    try:
        prompt_list = ast.literal_eval(agent_output)
    except (SyntaxError, ValueError) as e:
        # If both JSON and literal_eval fail, extract prompts using regex
        prompt_pattern = r'"(.*?)"'  # Matches text within double quotes
        prompt_list = re.findall(prompt_pattern, agent_output)
        if not prompt_list:
            raise ValueError(f"Failed to extract prompts using regex: {e}")

    if not isinstance(prompt_list, list):
        raise ValueError("Parsed output is not a list.")

except Exception as e:
    raise ValueError(f"Failed to parse prompts from agent: {e}")


print(f"✅ Got {len(prompt_list)} prompts from agent!")

# === 🎨 STEP 3: Generate and save images ===
client = openai.OpenAI(
    api_key= userdata.get("OPENAI_API_KEY")
)
for idx, prompt in enumerate(prompt_list):
    try:
        print(f"[{idx+1}/{len(prompt_list)}] Generating image for prompt: {prompt}")

        response = client.images.generate(
            model="dall-e-2",  # or "dall-e-2"
            prompt=prompt,
            size="256x256",
            quality="standard",
            n=1
        )

        image_url = response["data"][0]["url"]
        image_data = requests.get(image_url).content
        file_path = os.path.join(output_dir, f"image_{idx+1:02}.png")

        with open(file_path, 'wb') as f:
            f.write(image_data)

        print(f"✅ Saved: {file_path}")
        sleep(1)

    except Exception as e:
        print(f"❌ Error with prompt {idx+1}: {e}")


Enter a scenario (or type 'exit' to quit): trees
```python
["A towering redwood sequoia bathed in dappled sunlight, towering over a lush forest floor.", 
"A lone, gnarled oak tree standing sentinel on a windswept hilltop, branches reaching towards the stormy sky.",
"A grove of ancient olive trees, their silvery leaves shimmering in the Mediterranean sun.",
"A vibrant Japanese maple, ablaze with autumn colors, leaves cascading down like fiery rain.",
"Twisted and ancient cypress trees lining a foggy Italian path, creating an eerie and mysterious atmosphere.",
"A young sapling, its delicate leaves unfurling in the spring breeze, surrounded by wildflowers.",
"A dense forest of towering pine trees, their needles carpeting the ground in a thick layer of green.",
"A panoramic view of a sprawling forest, sunlight filtering through the leaves, creating a mosaic of light and shadow.",
"A close-up shot of a tree bark, showcasing its intricate patterns and textures.",
"A cluster of birch trees, t

KeyboardInterrupt: 

In [5]:
!pip install langgraph langsmith langchain langchain_groq langchain_community langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 28.4 MB/s eta 0:00:00


In [6]:
# Again new workflow

from typing import TypedDict, Dict, List, Any, Optional, Literal
# from fastapi import FastAPI, HTTPException, Body, Depends
# from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel, Field
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_groq import ChatGroq
from langchain_openai import OpenAI
import openai
from langgraph.graph import StateGraph, END
import re
import os
import json
from contextlib import asynccontextmanager

In [7]:
class GraphState(TypedDict):
    messages: List[Any]  # The messages passed between user and assistant
    conversation_history: Dict  # A record of all conversation threads
    current_agent: str  # Track which agent is currently active
    prompt_list: List[str]  # Store generated prompts
    sample_images: List[str]  # Store URLs of sample images
    user_feedback: str  # Store user feedback on sample images
    dataset_ready: bool  # Flag to indicate if user wants the full dataset
    llm_model: str  # Track which LLM model is being used
    session_id: str  # Unique identifier for the conversation session
    scenario: str   # We'll store the scenarion and update it accordingly

In [8]:
# Global variables to store active sessions
active_sessions = {}

# Agent initialization
def initialize_agents(groq_api_key, openai_api_key):
    """Initialize all the agent components"""
    conversation_agent = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
    prompt_generator = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")
    image_client = openai.OpenAI(api_key=openai_api_key)

    # Router agent
    router_agent = ChatGroq(groq_api_key=groq_api_key, model="Gemma2-9b-It")

    return conversation_agent, prompt_generator, image_client, router_agent


 Helper Functions

In [10]:
def extract_scenario(state):
    """Extract the user's scenario from the conversation history"""
    relevant_messages = []

    for message in state["messages"]:
        if isinstance(message, HumanMessage):
            relevant_messages.append(message.content)

    # Concatenate all user messages into a single scenario description
    if relevant_messages:
        combined_scenario = " ".join(relevant_messages)
        # If too long, use only the most substantial messages
        if len(combined_scenario) > 500:
            # Find the longest message, which likely contains the main scenario
            longest_message = max(relevant_messages, key=len)
            return longest_message
        return combined_scenario

    return "generic dataset of various objects and scenes"


def get_current_user_question(state):
    """Extract the most recent user question from the state"""
    for message in reversed(state["messages"]):
        if isinstance(message, HumanMessage):
            return message.content
    return ""


def parse_image_prompts(response_text):
    """Extract a list of prompts from the LLM response"""
    try:
        # Try to find a Python list in the text
        match = re.search(r'\[(.*?)\]', response_text, re.DOTALL)
        if match:
            prompts_text = match.group(1)
            # Safely evaluate the list
            return eval(f"[{prompts_text}]")

        # If no list is found, try to extract numbered items
        lines = response_text.split('\n')
        prompts = []
        for line in lines:
            # Look for numbered lines like "1. prompt" or "1) prompt"
            match = re.match(r'^\s*\d+[\.\)]\s*(.*)', line)
            if match:
                prompts.append(match.group(1).strip())

        if prompts:
            return prompts

        # If all else fails, split by newlines and filter
        return [line.strip() for line in lines if line.strip() and len(line.strip()) > 10]

    except Exception as e:
        print(f"Error parsing prompts: {e}")
        return []


In [20]:
def updated_scenario(state):
  scenario = state['scenario']
  scenario_update_llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")

  system = f""" You have a scenario which will be used to generate diverse prompts each will be used for
  for image generation. What your task is I will provide you initial scenario and you have to update it
  accordingly to the user instructions. You will have both user instructions and the scenario, return the updated scenario please.
  Scenario: {scenario}
  Instruction from User: {state['messages'][-1]}
  """
  # Update the prompt based on the given user instruction
  scenario_update_prompt = ChatPromptTemplate.from_messages(
      [
          ("system", system),
          ("human", "{question}"),
      ]
  )

  scenario_update_agent = scenario_update_prompt | scenario_update_llm
  response = scenario_update_agent.invoke(state['messages'][-1])

  #state['scenario'] = response.content
  return response.content


# LLM Router

In [15]:
def create_router(router_agent):
    """Create an LLM router to determine which agent should handle the request"""

    # Use JsonOutputParser to format LLM output
    parser = JsonOutputParser(pydantic_object=RouterOutput)

    system = """You are an expert routing agent for an image dataset creation system.
    Your job is to analyze the user's message and determine which of the following agents should handle it:

    1. conversation_agent: For general conversation, questions about the system, or when the user is still describing their needs.

    2. sample_image_agent: When the user is explicitly or implicitly requesting sample images, wants to see examples,
       or is describing specific image requirements that would benefit from visual confirmation before proceeding.

    3. dataset_generator_agent: When the user is ready to generate the full dataset of images, expressing satisfaction with samples,
       or directly asking to proceed with dataset creation.

    Analyze the content, context, and intent of the user's message, then select the most appropriate agent.

    Output your decision as a JSON object with the following fields:
    - agent: The name of the agent (one of the three above)
    - confidence: Your confidence level in this decision (0.0 to 1.0)
    - reasoning: A brief explanation of why you chose this agent
    """

    router_prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", "User message: {question}\n\nConversation history summary: {history_summary}")
    ])

    router_chain = router_prompt | router_agent | parser

    return router_chain


In [11]:
# Generate a summary of conversation history
def summarize_conversation(state):
    """Create a brief summary of the conversation history for context"""
    history = []

    # Extract last few turns of conversation
    messages = state["messages"][-10:]  # Limit to last 10 messages

    for msg in messages:
        if isinstance(msg, HumanMessage):
            history.append(f"User: {msg.content}")
        elif isinstance(msg, AIMessage):
            # Truncate very long assistant responses
            content = msg.content
            if len(content) > 200:
                content = content[:197] + "..."
            history.append(f"Assistant: {content}")

    return "\n".join(history)


# Routing function
def route_question(state: GraphState, router_chain):
    """Route the user question to the appropriate agent"""

    question = get_current_user_question(state)
    history_summary = summarize_conversation(state)

    # Make routing decision
    try:
        route_decision = router_chain.invoke({
            "question": question,
            "history_summary": history_summary
        })

        print(f"Router Decision: {route_decision}")

        # Update state to track which agent was selected
        state["current_agent"] = route_decision.agent

        # Set dataset_ready flag if we're routing to dataset generator
        if route_decision.agent == "dataset_generator_agent":
            state["dataset_ready"] = True

        # Map agent names to graph nodes
        agent_to_node = {
            "conversation_agent": "conversation",
            "sample_image_agent": "generate_sample_images",
            "dataset_generator_agent": "generate_prompts"
        }

        return agent_to_node[route_decision.agent]

    except Exception as e:
        print(f"Routing error: {e}")
        # Default to conversation as fallback
        return "conversation"

# Agent Handlers

In [21]:
def conversation_handler(state: GraphState):
    """The primary conversation agent to interact with the user"""

    user_message = state["messages"][-1].content

    system_message = """You are an AI assistant specializing in helping users create image datasets.
    Your role is to understand what kind of image dataset the user wants to create.

    Be conversational and helpful, focusing on understanding the user's requirements clearly.
    If the user seems to be describing a specific type of images, suggest they might want to see samples.
    If they've seen samples and like them, you can suggest generating the full dataset.

    Keep your responses concise and focused on helping create their image dataset.
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message),
        ("human", "{user_input}")
    ])

    conversation_agent = state["conversation_agent"]
    chain = prompt | conversation_agent
    response = chain.invoke({"user_input": user_message})

    # Add to conversation history
    conversation_id = len(state["conversation_history"]) + 1
    state["conversation_history"][conversation_id] = {
        "question": user_message,
        "response": response
    }

    state["messages"].append(AIMessage(content=response.content))
    state["current_agent"] = "conversation_agent"
    state['scenario'] = updated_scenario(state)

    return "router"


def sample_image_handler(state: GraphState):
    """Generate sample images for user review"""

    # Extract user scenario from conversation
    user_scenario = state['scenario']

    # Generate a few sample prompts
    system = """You are a prompt generator for text-to-image models.
    Generate 3 diverse and detailed prompts for sample images based on the user scenario.
    Each prompt should be descriptive and optimized for image generation.
    Format your output as a Python list of exactly 3 strings: ["prompt1", "prompt2", "prompt3"]
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", f"Generate 3 sample image prompts for scenario: {user_scenario}")
    ])

    prompt_generator = state["prompt_generator"]
    chain = prompt | prompt_generator
    response = chain.invoke({})

    # Extract the prompt list
    sample_prompts = parse_image_prompts(response.content)

    # Ensure we have exactly 3 prompts
    if len(sample_prompts) < 3:
        # Fill in missing prompts
        while len(sample_prompts) < 3:
            sample_prompts.append(f"{user_scenario} variation {len(sample_prompts)+1}")
    elif len(sample_prompts) > 3:
        # Trim to just 3 prompts
        sample_prompts = sample_prompts[:3]

    # Generate sample images
    sample_image_urls = []
    for prompt in sample_prompts:
        try:
            response = state["image_client"].images.generate(
                model="dall-e-2",
                prompt=prompt,
                size="256x256",
                quality="standard",
                n=1
            )
            sample_image_urls.append(response.data[0].url)
        except Exception as e:
            print(f"Error generating image: {str(e)}")
            sample_image_urls.append(None)

    # Filter out any None values
    sample_image_urls = [url for url in sample_image_urls if url is not None]

    state["sample_images"] = sample_image_urls
    state["prompt_list"] = sample_prompts

    # Create response message with sample images
    response_text = "Here are some sample images based on your requirements:\n\n"
    for i, (prompt, url) in enumerate(zip(sample_prompts, sample_image_urls)):
        if url:
            response_text += f"Sample {i+1}:\nPrompt: {prompt}\n\n"

    response_text += "How do these look? Would you like to make any adjustments before generating the full dataset of 50 images?"

    state["messages"].append(AIMessage(content=response_text))
    state["current_agent"] = "sample_image_agent"

    return "router"



In [17]:
def prompt_generator_handler(state: GraphState):
    """Generate the full set of prompts based on user requirements"""

    # Extract user scenario from conversation
    user_scenario = extract_scenario(state)
    user_feedback = state.get("user_feedback", "")

    # Include user feedback if available
    scenario_with_feedback = user_scenario
    if user_feedback:
        scenario_with_feedback = f"{user_scenario}. User feedback on samples: {user_feedback}"

    system = """You are a prompt generator for text to image models. Your task is to generate 50 diverse prompts
    for text-to-image models to create a comprehensive dataset given the scenario.
    Each prompt should be descriptive and optimized for image generation.
    Make sure to cover different angles, perspectives, environments, and contexts.
    Format your output as a Python list of strings: ["prompt1", "prompt2", ...]
    """

    prompt = ChatPromptTemplate.from_messages([
        ("system", system),
        ("human", f"Generate 50 diverse image prompts for scenario: {scenario_with_feedback}")
    ])

    prompt_generator = state["prompt_generator"]
    chain = prompt | prompt_generator
    response = chain.invoke({})

    # Extract the prompt list
    prompt_list = parse_image_prompts(response.content)

    # Ensure we have at least 50 prompts
    if len(prompt_list) < 50:
        # Fill in missing prompts
        while len(prompt_list) < 50:
            prompt_list.append(f"{user_scenario} variation {len(prompt_list)+1}")

    state["prompt_list"] = prompt_list[:50]  # Limit to exactly 50 prompts
    state["dataset_ready"] = True

    # Prepare response message
    response_text = f"I've generated 50 diverse prompts for your dataset based on your requirements.\n\n"
    response_text += "Here are a few examples:\n"
    for i, prompt in enumerate(prompt_list[:5]):
        response_text += f"{i+1}. {prompt}\n"

    response_text += f"\n...and {len(prompt_list)-5} more prompts.\n\n"
    response_text += "Would you like me to proceed with generating all 50 images for your dataset now?"

    state["messages"].append(AIMessage(content=response_text))
    state["current_agent"] = "dataset_generator_agent"

    return "router"


def dataset_generator_handler(state: GraphState):
    """Generate the full dataset of images"""

    prompt_list = state["prompt_list"][:50]  # Ensure we use max 50 prompts
    image_urls = []

    # Create a progress update message
    progress_message = "Starting to generate your dataset with 50 images. This might take some time...\n"
    state["messages"].append(AIMessage(content=progress_message))

    # Generate images for each prompt - for real implementation
    for idx, prompt in enumerate(prompt_list):
        try:
            print(f"[{idx+1}/{len(prompt_list)}] Generating image for prompt: {prompt}")

            response = state["image_client"].images.generate(
                model="dall-e-2",
                prompt=prompt,
                size="256x256",
                quality="standard",
                n=1
            )

            image_url = response.data[0].url
            image_urls.append(image_url)

        except Exception as e:
            error_msg = f"Error generating image {idx+1}: {str(e)}"
            print(error_msg)
            image_urls.append(None)

    # Filter out None values
    image_urls = [url for url in image_urls if url is not None]

    # Final response with dataset information
    final_message = f"✅ Dataset generation complete! Generated {len(image_urls)}/50 images.\n\n"

    # Include some sample URLs
    if image_urls:
        final_message += "Here are a few sample images from your dataset:\n"
        for i, url in enumerate(image_urls[:5]):
            if url:
                final_message += f"Image {i+1}: {url}\n"

    final_message += "\nYou can download these images to assemble your complete dataset. Would you like me to help with anything else?"

    state["messages"].append(AIMessage(content=final_message))
    state["current_agent"] = "dataset_generator_agent"

    return "router"
